In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import folium
import csv

# Recover Ripple Graph

In [2]:
def recover_links_ases(filename):
    links = set()
    ases = set()
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            l = tuple(row)
            links.add(l)
            ases.add(l[0])
            ases.add(l[1])
    return ases,links

In [3]:
ripple_ases, ripple_links = recover_links_ases('Ripple/gateway_links.csv')

In [4]:
ripple_graph = nx.Graph()
ripple_graph.add_nodes_from(ripple_ases)

In [5]:
print('The Ripple graph contains {} nodes'.format(len(ripple_graph.nodes)))

The Ripple graph contains 15 nodes


# Recover Caida Graph

In [6]:
data = pd.read_csv('Caida/20190301.as-rel2.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

data['provider_peer'], data['customer_peer'], data['relation_type'], data['source'] = data[0].str.split('|').str
data['relation_type']  = data['relation_type'].apply(lambda x: 'provider-customer' if x == '-1' else 'peer-peer')
links = data[['provider_peer','customer_peer','relation_type']]

caida_ases = set(links['provider_peer'].values)
caida_ases.update(links['customer_peer'].values)

links['link'] = list(zip(links['provider_peer'], links['customer_peer']))
links = links[['link','relation_type']]
caida_links = {}
for index,row in links.iterrows():
    (s,t) = row['link']
    rel = row['relation_type']
    if rel == 'provider-customer':
        caida_links[tuple([s,t])] = rel
        caida_links[tuple([t,s])] = 'customer-provider'
    else: 
        if rel == 'peer-peer':
            caida_links[tuple([s,t])] = rel
            caida_links[tuple([t,s])] = 'peer-peer'
        else:
            print('Error')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
caida_graph = nx.Graph()
caida_graph.add_nodes_from(caida_ases)
for k,v in caida_links.items():
    caida_graph.add_edge(k[0],k[1])

In [8]:
print('The Ripple graph contains {} nodes and {} edges'.format(len(caida_graph.nodes),len(caida_graph.edges)))

The Ripple graph contains 64167 nodes and 503651 edges


# Add intermediary nodes to Ripple Graph

In [9]:
def shortest_paths(s,t,G):
    actual_paths = []
    paths = nx.all_shortest_paths(G,s,t)
    for path in paths:
        actual_path = []
        for i in range(len(path)-1):
            l = tuple([path[i],path[i+1]])
            actual_path.append(l)
        actual_paths.append(actual_path)
    return actual_paths

In [10]:
for s in ripple_ases:
    for t in ripple_ases:
        paths = nx.all_shortest_paths(caida_graph,s,t)
        for p in paths:
            complete_path = [s] + p
            for i in range(len(complete_path)-1):
                ripple_graph.add_edge(complete_path[i],complete_path[i+1])

In [11]:
print('The Ripple graph contains {} nodes and {} edges'.format(len(ripple_graph.nodes),len(ripple_graph.edges)))

The Ripple graph contains 324 nodes and 1299 edges


In [12]:
nx.set_edge_attributes(ripple_graph, 0, 'weight')

# Replay transactions 

In [13]:
transactions = pd.read_csv('Ripple/transactions_processed.csv')
transactions = transactions[['sender','receiver','amount']]

In [14]:
for index, row in transactions.iterrows():
    sender = str(row['sender'])[:-2]
    receiver = str(row['receiver'])[:-2]
    amount = row['amount']
    
    try :
        ## For the moment, use only the best path
        path = nx.shortest_path(ripple_graph,sender,receiver)
        for i in range(len(path)-1):
            s_p = path[i]
            t_p = path[i+1]
            ripple_graph[s_p][t_p]['weight'] += amount
    except:
        print('No path between {} and {}'.format(sender,receiver))

In [17]:
for d in ripple_graph.degree(weight='weight'):
    if d[0] in ripple_ases:
        print('The following node is a Ripple node:')
    print('AS: {} - Money: {}'.format(d[0],d[1]))
    print()

The following node is a Ripple node:
AS: 46606 - Money: 0

The following node is a Ripple node:
AS: 14618 - Money: 19763467.82353659

The following node is a Ripple node:
AS: 60781 - Money: 6562.246228113321

The following node is a Ripple node:
AS: 13335 - Money: 16205331.87731691

The following node is a Ripple node:
AS: 38895 - Money: 19598728.550145373

The following node is a Ripple node:
AS: 57127 - Money: 3.51606274

The following node is a Ripple node:
AS: 9371 - Money: 790.559258553264

The following node is a Ripple node:
AS: 16509 - Money: 39560636.26510259

The following node is a Ripple node:
AS: 26496 - Money: 44.10200000000001

The following node is a Ripple node:
AS: 4766 - Money: 0

The following node is a Ripple node:
AS: 53667 - Money: 0.047201516516510965

The following node is a Ripple node:
AS: 37907 - Money: 0.0002984829932322704

The following node is a Ripple node:
AS: 19551 - Money: 6742692.895961824

The following node is a Ripple node:
AS: 45102 - Money: 544